# Evaluating your first agent

In this tutorial, we'll build upon the web search agent from [my_first_agent.ipynb](https://github.com/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) and demonstrate how to evaluate its performance using any-agent's evaluation framework. We'll explore different evaluation methods including custom code evaluation, an LLM-based judge, and an agent-based judge. 

Note: Since we are building on the previous notebook, we encourage you to run that one first to read through details and choices available while building the agent before evaluating it.


## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Set Up the Web Search Agent

First, let's recreate the web search agent from [the previous tutorial](https://github.com/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) so we have something to evaluate.

In [3]:
import os
from getpass import getpass

if "MISTRAL_API_KEY" not in os.environ:
    print("MISTRAL_API_KEY not found in environment!")
    api_key = getpass("Please enter your MISTRAL_API_KEY: ")
    os.environ["MISTRAL_API_KEY"] = api_key
    print("MISTRAL_API_KEY set for this session!")
else:
    print("MISTRAL_API_KEY found in environment.")

MISTRAL_API_KEY found in environment.


In [4]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import search_web, visit_webpage

agent = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="mistral/mistral-small-latest", tools=[search_web, visit_webpage]
    ),
)

## Run the Agent to Generate a Trace

Now let's run our agent on a test query to generate a trace that we can evaluate.

In [5]:
prompt = "What film won a Goya Award for best film in 2024? Please search for the most relevant page, visit it and provide the name of the film, the genre, and a brief description of the film in under 10 words."

agent_trace = agent.run(prompt)

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You are an agent - please keep going until the user's query is completely resolved, before  │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What film won a Goya Award for best film in 2024? Please search for the most relevant page, │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_web",                                                                              │ │
│ │     "tool.args": "{\"query\": \"Goya Award best film 2024\"}"                                               │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 446,                                                                                      │ │
│ │   "output_tokens": 20,                                                                                      │ │
│ │   "input_cost": 4.46e-05,                                                                                   │ │
│ │   "output_cost": 6e-06                                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── EXECUTE_TOOL: search_web ────────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Goya Award best film 2024"                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ ]8;id=371853;https://en.wikipedia.org/wiki/38th_Goya_Awards\38th Goya Awards - Wikipedia]8;;\ The 38th Goya Awards ceremony, presented by the Academy of Cinematographic     │ │
│ │ Arts and Sciences of Spain, took place at the Feria de Valladolid [es] premises in Valladolid, Castile and  │ │
│ │ León on 10 February 2024. [2] The ceremony was broadcast on La 1 and RTVE Play. [3] J. A. Bayona 's Society │ │
│ │ of the Snow received the most awards during the ceremony with twelve wins, including Best Film. [4] ]8;id=551032;https://deadline.com/2024/02/goya-awards-complete-winners-list-1235821487/\Goya ]8;;\   │ │
│ │ ]8;id=23778;https://deadline.com/2024/02/goya-awards-complete-winners-list-1235821487/\Awards Winners: 'The Society Of The Snow' Takes Best Picture ...]8;;\ Spanish cinema was celebrated at the 38th  │ │
│ │ Annual Goya Awards in Valladolid, with Netflix's The Society of the Snow taking a total of 12 trophies, the │ │
│ │ most of the night. ]8;id=388418;https://highxtar.com/en/complete-list-of-the-winners-of-the-goya-awards-2024/\Complete list of the winners of the Goya Awards 2024]8;;\ The 38th edition of the Goya Awards │ │
│ │ was held this Saturday 10 February in Valladolid. The big winner of the night was La sociedad de la nieve,  │ │
│ │ with 12 awards, making it the third most awarded film in history. Among the awards are Best Film, Best      │ │
│ │ Director and Best New Actor for Matías Recalt, and Best Soundtrack, which were not foreseen in the pools,   │ │
│ │ as well as all the technical awards. Best ... ]8;id=734624;https://variety.com/2024/awards/global/j-a-bayona-penelope-cruz-1235907010/\Spain's Goya Awards: 'The Society of the Snow,' Sigourney ]8;;\    │ │
│ │ ]8;id=272971;https://variety.com/2024/awards/global/j-a-bayona-penelope-cruz-1235907010/\Weaver]8;;\ Netflix Original "The Society of the Snow" won best picture and director for J.A. Bayona at Saturday │ │
│ │ night's 38th Spanish Academy Goya Awards. Those plaudits were two of a total 12 prizes ... ]8;id=82821;https://www.musicmundial.com/en/2024/02/10/this-is-the-list-of-winners-of-the-goya-awards-2024/\This is the list]8;;\ │ │
│ │ ]8;id=15444;https://www.musicmundial.com/en/2024/02/10/this-is-the-list-of-winners-of-the-goya-awards-2024/\of winners of the Goya Awards 2024 - Music Mundial News]8;;\ This Saturday, February 10, Valladolid hosted the   │ │
│ │ big night of Spanish cinema, the night of the Goya Awards 2024. Ana Belén and Ambrossi and Calvo presented  │ │
│ │ the gala. The following is the list of winners at the 38th edition of the Goya Awards, presented in         │ │
│ │ Valladolid, Spain: -Film: "Snow Society ... ]8;id=140054;https://www.latintimes.com/goya-awards-2024-la-sociedad-de-la-nieve-skis-away-top-honors-spanish-oscars-551214\Goya Awards 2024: 'La Sociedad de la Nieve' Skis Away with Top ]8;;\ │ │
│ │ ]8;id=579691;https://www.latintimes.com/goya-awards-2024-la-sociedad-de-la-nieve-skis-away-top-honors-spanish-oscars-551214\Honors ...]8;;\ At the Goya Awards, "La sociedad de la nieve" achieved an astounding victory, securing 10        │ │
│ │ awards, including Best Picture. ]8;id

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "visit_webpage",                                                                           │ │
│ │     "tool.args": "{\"url\": \"https://en.wikipedia.org/wiki/38th_Goya_Awards\"}"                            │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 1608,                                                                                     │ │
│ │   "output_tokens": 25,                                                                                      │ │
│ │   "input_cost": 0.0001608,                                                                                  │ │
│ │   "output_cost": 7.499999999999999e-06                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: visit_webpage ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "url": "https://en.wikipedia.org/wiki/38th_Goya_Awards"                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ 38th Goya Awards - Wikipedia ]8;id=106591;#bodyContent\Jump to content]8;;\ Main menu Main menu move to sidebar hide Navigation            │ │
│ │                                                                                                             │ │
│ │ 🌆 ]8;id=417866;/wiki/Main_Page\wikipedia.png]8;;\ 🌆 ]8;id=606290;/wiki/Main_Page\Wikipedia]8;;\ 🌆 ]8;id=933521;/wiki/Main_Page\The Free Encyclopedia]8;;\                                                      │ │
│ │  • ]8;id=338201;/wiki/Main_Page\Main page]8;;\                                                                                                │ │
│ │  • ]8;id=143599;/wiki/Wikipedia:Contents\Contents]8;;\                                                                                                 │ │
│ │  • ]8;id=291363;/wiki/Portal:Current_events\Current events]8;;\                                                                                           │ │
│ │  • ]8;id=1502;/wiki/Special:Random\Random article]8;;\                                                                                           │ │
│ │  • ]8;id=340968;/wiki/Wikipedia:About\About Wikipedia]8;;\                                                                                          │ │
│ │  • ]8;id=29365;//en.wikipedia.org/wiki/Wikipedia:Contact_us\Contact us]8;;\ Contribute                                                                                    │ │
│ │  • ]8;id=557543;/wiki/Help:Contents\Help]8;;\                                                                                                     │ │
│ │  • ]8;id=592635;/wiki/Help:Introduction\Learn to edit]8;;\                                                                                            │ │
│ │  • ]8;id=795315;/wiki/Wikipedia:Community_portal\Community portal]8;;\                                                                                         │ │
│ │  • ]8;id=834797;/wiki/Special:RecentChanges\Recent changes]8;;\                                                                                           │ │
│ │  • ]8;id=960038;/wiki/Wikipedia:File_upload_wizard\Upload file]8;;\                                                                                              │ │
│ │  • ]8;id=737922;/wiki/Special:SpecialPages\Special pages]8;;\ ]8;id=7710;/wiki/Main_Page\ ]8;;\]8;id=791185;/wiki/Main_Page\ ]8;;\ ]8;id=235110;/wiki/Special:Search\Search]8;;\ Search Appearance                                                                │ │
│ │  • ]8;id=793814;https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en\Donate]8;;\                                                                                                   │ │
│ │  • ]8;id=204904;/w/index.php?title=Special:CreateAccount&returnto=38th+Goya+Awards\Create account]8;;\                                                                                           │ │
│ │  • ]8;id=477082;/w/index.php?title=Special:UserLogin&returnto=38th+Goya+Awards\Log in]8;;\ Personal tools                                    

╭──────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The film that won the Goya Award for Best Film in 2024 is "La sociedad de la nieve" (Society of the Snow).  │ │
│ │                                                                                                             │ │
│ │ Genre: Survival drama                                                                                       │ │
│ │                                                                                                             │ │
│ │ Description: Survivors of a plane crash endure harsh conditions in the Andes.                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 4958,                                                                                     │ │
│ │   "output_tokens": 54,                                                                                      │ │
│ │   "input_cost": 0.0004958,                                                                                  │ │
│ │   "output_cost": 1.62e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## View the Agent Results

Let's first see what our agent produced:


In [6]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.tokens.total_tokens:,}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")

The film that won the Goya Award for Best Film in 2024 is "La sociedad de la nieve" (Society of the Snow).

Genre: Survival drama

Description: Survivors of a plane crash endure harsh conditions in the Andes.
Duration: 3.88 seconds
Usage: 7,111
Cost (USD): 0.000731


## Method 1: Custom Code Evaluation

Before using LLM-based evaluation, let's start with deterministic custom code evaluation. This is often more efficient, reliable, and cost-effective for specific criteria.

Some criteria are clearly quantitative: a result exists or it doesn't, it has a measurable length, the number of steps can be counted and a tool was either called or wasn't.

In [7]:
from any_agent.tracing.agent_trace import AgentTrace


def evaluate_web_search_efficiency(trace: AgentTrace) -> dict:
    """Custom evaluation function for web search agent efficiency criteria."""
    # Direct access to trace properties
    token_count = trace.tokens.total_tokens
    step_count = len(trace.spans)
    final_output = trace.final_output
    duration = trace.duration.total_seconds()
    # Check if web search tools were used
    messages = trace.spans_to_messages()
    used_search = any(
        message.role == "assistant" and "search_web" in str(message.content)
        for message in messages
    )
    used_visit = any(
        message.role == "assistant" and "visit_webpage" in str(message.content)
        for message in messages
    )
    # Apply quantitative criteria
    results = {
        "token_efficient": token_count
        < 20000,  # Magic number alert: adjust to what you consider reasonable for your budget
        "step_efficient": step_count
        <= 10,  # A high number of steps would point at problems, but this is also a debatable limit
        "has_output": final_output is not None and len(str(final_output)) > 5,
        "short_output": len(str(final_output)) < 10 if final_output else 0,
        "used_web_search": used_search,
        "used_webpage_visit": used_visit,
        "reasonable_duration": duration < 60,
    }
    # Choose the quantitative criteria you care most about
    results["passed"] = all(
        [
            results["token_efficient"],
            results["step_efficient"],
            results["has_output"],
            results["used_web_search"],
            results["short_output"],
        ]
    )
    return results

In [8]:
evaluation = evaluate_web_search_efficiency(agent_trace)
print("Custom Code Evaluation Results:")
for key, value in evaluation.items():
    print(f"  {key}: {value}")

Custom Code Evaluation Results:
  token_efficient: True
  step_efficient: True
  has_output: True
  short_output: False
  used_web_search: True
  used_webpage_visit: True
  reasonable_duration: True
  passed: False


## Method 2: LLM Judge Evaluation

The method above is already useful and can assess quantitative results (how long or how costly answers were, whether a specific tool was present). Programmatic evaluations are less costly, more deterministic, but also less flexible.
For more qualitative assessments, you can also use the `LlmJudge`. This is great for evaluating response quality, helpfulness, and other subjective criteria.

### 💡 Good to know: different models

Notice we use a different LLM as a judge to the one we used for the original agent, as LLM judges are known to have a [bias towards their own results](https://neurips.cc/virtual/2024/poster/96672).

In [9]:
from any_agent.evaluation import LlmJudge

# Create an LLM judge
judge = LlmJudge(model_id="gpt-4.1-mini")

# Define evaluation questions
evaluation_questions = [
    "Did the agent provide a clear and concise answer?",
    "Did the agent correctly identify the genre?",
    "Did the agent provide a brief description (under 10 words) of the film?",
    "Did the agent use web search tools to gather current information?",
    "Was the agent's whole output, structured or not, metadata or no, everything added together, shorter than 10 words?",
]

# Run evaluations
print("LLM Judge Evaluation Results:")
print("=" * 60)

results = []
for i, question in enumerate(evaluation_questions, 1):
    result = judge.run(context=str(agent_trace.spans_to_messages()), question=question)
    results.append(result)
    print(f"Question {i}: {question}")
    print(f"  Passed: {result.passed}")
    print(f"  Reasoning: {result.reasoning}")
    print("-" * 60)

# Summary
passed_count = sum(1 for r in results if r.passed)
print(f"\nOverall: {passed_count}/{len(results)} criteria passed")

LLM Judge Evaluation Results:
Question 1: Did the agent provide a clear and concise answer?
  Passed: True
  Reasoning: The agent clearly identified the film that won the Goya Award for Best Film in 2024 as "La sociedad de la nieve" (Society of the Snow). The agent also provided the genre as "Survival drama" and gave a brief description of the film in under 10 words: "Survivors of a plane crash endure harsh conditions in the Andes." This satisfies the user's request for the film's name, genre, and a concise description, all presented clearly and without unnecessary information.
------------------------------------------------------------
Question 2: Did the agent correctly identify the genre?
  Passed: True
  Reasoning: The agent identified the film that won the Goya Award for Best Film in 2024 as "La sociedad de la nieve" (Society of the Snow) and stated the genre as "Survival drama." This matches the thematic description of the film about survivors of a plane crash enduring harsh con

### 💡 Good to know: fuzzy criteria 

Notice Question 3: if you run the evaluation multiple times, it won't pass or fail consistently, since the LLMJudge may interpret that only the description should be under 10 words, not necessarily the whole Agent's answer. In the programmatic method, there is nothing to interpret: we check that the final output was under 10 words. 

For comparison, look at Question 5, where we are even more specific in what our criterion is for the Agent to pass. Results (pass or fail) should be much more consistent in this case.

This showcases the main downside with using an LLMJudge: as with humans, criteria can be misunderstood.

## Method 3: Agent Judge Evaluation

For more complex evaluations that require inspecting specific aspects of the trace, we can use the `AgentJudge`. Notice the AgentJudge can:

* call built-in tools to get straight to relevant parts of the traces (e.g. final output),
* call additional tools that the original agent did not have. For example, you will see below how we give it a second search tool so it can do its own research to check if the original agent's answer was correct.

As with the LLMJudge, we choose a different model to the one enabling the original judge.

Notice, if you have a [Tavily API key](https://www.tavily.com/), you can import and use `search_tavily` as an alternative to `search_web`.

In [10]:
from any_agent.evaluation import AgentJudge
from any_agent.tools import search_web

# Create an agent judge
agent_judge = AgentJudge(model_id="gpt-4.1-mini")

# Define a complex evaluation question that requires trace inspection
complex_question = """
Evaluate the agent's performance on this web search task by considering:
1. Did the agent use the search_web tool to find relevant information?
2. Did the agent visit a webpage to get detailed content?
3. Did the final answer contain both the film's original name and its genre?
4. Was the full output of the agent, not just a specific section or answer, but the whole text it produced as final output, within the requested constraints (under 10 words)?
5. Did the agent demonstrate good web search practices?

Use the available tools to inspect the trace and provide a comprehensive evaluation, specially for the final output. Additionally, use the search tool to verify the agent's answer to the original question: 'who won the Goya Award for best film in 2024'.
"""

# Run the agent judge evaluation
eval_trace = agent_judge.run(
    trace=agent_trace,
    question=complex_question,
    additional_tools=[
        search_web
    ],  # Give the judge access to web search for verification
)

# Get the evaluation result
result = eval_trace.final_output
print("Agent Judge Evaluation Result:")
print("=" * 60)
print(f"Passed: {result.passed}")
print(f"Reasoning: {result.reasoning}")
print("=" * 60)

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You are a helpful assistant that will be used to evaluate the correctness of an agent trace │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "\nEvaluate the agent's performance on this web search task by considering:\n1. Did the agen │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_steps_taken",                                                                         │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_final_output",                                                                        │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_messages_from_trace",                                                                 │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 593,                                                                                      │ │
│ │   "output_tokens": 54,                                                                                      │ │
│ │   "input_cost": 0.0002372,                                                                                  │ │
│ │   "output_cost": 8.64e-05                           

╭───────────────────────────────────────── EXECUTE_TOOL: get_steps_taken ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ 6                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── EXECUTE_TOOL: get_final_output ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The film that won the Goya Award for Best Film in 2024 is "La sociedad de la nieve" (Society of the Snow).  │ │
│ │                                                                                                             │ │
│ │ Genre: Survival drama                                                                                       │ │
│ │                                                                                                             │ │
│ │ Description: Survivors of a plane crash endure harsh conditions in the Andes.                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── EXECUTE_TOOL: get_messages_from_trace ─────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │                                                   system                                                    │ │
│ │                                                                                                             │ │
│ │ You are an agent - please keep going until the user's query is completely resolved, before ending your turn │ │
│ │ and yielding back to the user. Only terminate your turn when you are sure that the problem is solved, or if │ │
│ │ you need more info from the user to solve the problem.                                                      │ │
│ │                                                                                                             │ │
│ │ If you are not sure about anything pertaining to the user's request, use your tools to read files and       │ │
│ │ gather the relevant information: do NOT guess or make up an answer.                                         │ │
│ │                                                                                                             │ │
│ │ You MUST plan extensively before each function call, and reflect extensively on the outcomes of the         │ │
│ │ previous function calls. DO NOT do this entire process by making function calls only, as this can impair    │ │
│ │ your ability to solve the problem and think insightfully.                                                   │ │
│ │                                                                                                             │ │
│ │                                                    user                                                     │ │
│ │                                                                                                             │ │
│ │ What film won a Goya Award for best film in 2024? Please search for the most relevant page, visit it and    │ │
│ │ provide the name of the film, the genre, and a brief description of the film in under 10 words.             │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [{"tool.name": "search_web", "tool.args": "{"query": "Goya Award best film 2024"}"}]                        │ │
│ │                                                                                                             │ │
│ │                                                  assistant                                                  │ │
│ │                                                                                                             │ │
│ │ [Tool search_web executed: ]8;id=825820;https://en.wikipedia.org/wiki/38th_Goya_Awards\38th Goya Awards - Wikipedia]8;;\ The 38th Goya Awards ceremony, presented by the     │ │
│ │ Academy of Cinematographic Arts and Sciences of Spain, took place at the Feria de Valladolid [es] premises  │ │
│ │ in Valladolid, Castile and León on 10 February 2024. [2] The ceremony was broadcast on La 1 and RTVE Play.  │ │
│ │ [3] J. A. Bayona 's Society of the Snow received the most awards during the ceremony with twelve wins,      │ │
│ │ including Best Film. [4] ]8;id=160022;https://deadline.com/2024/02/goya-awards-complete-winners-li

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "search_web",                                                                              │ │
│ │     "tool.args": "{\"query\":\"Goya Award for best film in 2024\"}"                                         │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 5604,                                                                                     │ │
│ │   "output_tokens": 23,                                                                                      │ │
│ │   "input_cost": 0.0022416,                                                                                  │ │
│ │   "output_cost": 3.68e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── EXECUTE_TOOL: search_web ────────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "Goya Award for best film in 2024"                                                               │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Error calling tool: https://html.duckduckgo.com/html 202 Ratelimit                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "passed": false,                                                                                          │ │
│ │   "reasoning": "The agent did use the search_web tool and visited a relevant Wikipedia page for detailed co │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 5653,                                                                                     │ │
│ │   "output_tokens": 129,                                                                                     │ │
│ │   "input_cost": 0.0022612,                                                                                  │ │
│ │   "output_cost": 0.0002064                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "passed": false,                                                                                          │ │
│ │   "reasoning": "The agent used the search_web tool and visited a relevant webpage to obtain detailed inform │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 5643,                                                                                     │ │
│ │   "output_tokens": 94,                                                                                      │ │
│ │   "input_cost": 0.0022572,                                                                                  │ │
│ │   "output_cost": 0.0001504                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Agent Judge Evaluation Result:
Passed: False
Reasoning: The agent used the search_web tool and visited a relevant webpage to obtain detailed information, fulfilling points 1 and 2. The final answer included the film's original name and its genre, meeting point 3. However, the final answer exceeded the requested constraint to be under 10 words, failing point 4. The agent showed good web search practices by searching multiple sources and verifying the information, meeting point 5.


Notice here again how hard it is to make the agent understand the length requirements. Do play around with them and notice how the assessment changes!